<a href="https://colab.research.google.com/github/clarrabureg/testrepo/blob/main/Tarea_Final_Bat_Vecindarios_Lima_parte2_CLLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------------------------------------------

## Tarea Final COURSERA -  Batalla de vecindarios de LIMA - PERU Parte 2 - CLLG
-------------------------------------------------------------

**Indicaciones del Trabajo Final - Parte 2 (semana2)** 
Para la segunda semana, las entregas finales del proyecto serán:
*  Un enlace a su Notebook en su repositorio de Github, que muestra su código. (15 puntos)
*  Un informe completo que consta de todos los siguientes componentes (15 puntos):
* Introducción en la que discute el problema comercial y quién estaría interesado en este proyecto.
* Datos donde describe los datos que se utilizarán para resolver el problema y la fuente de los datos.
* Sección de metodología que representa el componente principal del informe en el que analiza y describe cualquier análisis de datos exploratorio que realizó, cualquier prueba estadística inferencial que realizó, si corresponde, y qué aprendizaje automático se utilizó y por qué.
*Sección de resultados donde discute los resultados.
*Sección de debate en la que analiza las observaciones que notó y las recomendaciones que pueda hacer en función de los resultados.
*Sección de conclusiones donde concluye el informe.
*Su elección de presentación o publicación de blog. (10 puntos)


---
INFORMACION DEL TRABAJO PREVIO REALIZADO
     

1.   Los archivos de datos fueron limpiados y preparados  mediante los programas:
        *   TRANS_GEO_Batalladevecindarios1.ipynb
        *   TRANS_UBIGEO_Batalladevecindarios1.ipynb
        *   COMBINA_Batalladevecindarios1.ipynb
2.   Se generó la tabla Newtabla.csv en donde se encuentra     ya List item la información lista para realizar los mapas y   agrupaciones
3.  Se ejecutó la generacion de mapas como parte del trabajo de la semana 1 mediante el programa:
       * Tarea_Final_Bat_Vecind_Lima_parte_1-CLLG .ipynb
---

### 1. Aplicación de la plataforma Foursquare

Como indicamos en la 1era. parte del trabajo, seleccionaremos para trabajar con Foursquare los datos de los distritos que tienen un mayor porcentaje de poblacion del nivel socioeconomico AB.

------------------------------------------------------
#### 2. PREPARACION DEL ENTORNO DE TRABAJO 
---

In [1]:
import pandas as pd
import numpy as np
import json # librería para manejar archivos JSON 
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud
import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas
# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors
# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 
from folium.features import DivIcon


-----------------------------------------------------------

## 3. CARGA DEL ARCHIVO DE DATOS PREPARADO PARA ESTE TRABAJO
-----------------------------------------------------------

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# CONSIDERAR CARGAR EL ARCHIVO EN JUPYTER O COLAB Y CAMBIAR LA RUTA SI NO LO CORREN CON LOCAL COMO EN ESTE CASO
# SI EL ARCHIVO ES EN CSV USA ESTA LINEA
df_original = pd.read_csv("/content/drive/MyDrive/labatalla/newtabla.csv", sep = ",")
# SI EL ARCHIVO ES EN EXCEL USA ESTA LINEA 
#df_original = pd.read_excel("/content/drive/MyDrive/labatalla/GeoPeru-peru_distritos.xlsx")

Mounted at /content/drive


In [3]:
# Presenta la tabla que contiene solo los distritos que superan el 20% de poblacion con NSE > 20%
df_original.head(50)


,Unnamed: 0,id_ubigeo,departamento_x,provincia_x,distrito,region,altitud,latitud,longitud,porcentaje_AB,total_pers,num_hog,pob_total
0,11,1320,LIMA,LIMA,SANTIAGO DE SURCO,LIMA PROVINCIA,107.0,-12.1450,-77.0050,78.9,325101,100585,329152
1,21,1294,LIMA,LIMA,LA MOLINA,LIMA PROVINCIA,262.0,-12.0781,-76.9167,78.9,139654,40245,140679
2,30,1310,LIMA,LIMA,SAN BORJA,LIMA PROVINCIA,170.0,-12.1072,-76.9989,78.9,111015,34940,113247
3,24,1302,LIMA,LIMA,MIRAFLORES,LIMA PROVINCIA,125.0,-12.1217,-77.0292,78.9,97322,37626,99337
4,39,1311,LIMA,LIMA,SAN ISIDRO,LIMA PROVINCIA,195.0,-12.0978,-77.0272,78.9,59872,21073,60735
5,32,1316,LIMA,LIMA,SAN MIGUEL,LIMA PROVINCIA,84.0,-12.0922,-77.0794,70.2,152265,46210,155384
6,29,1301,LIMA,LIMA,PUEBLO LIBRE,LIMA PROVINCIA,114.0,-12.0781,-77.0625,70.2,82756,26407,83323
7,25,1293,LIMA,LIMA,JESUS MARIA,LIMA PROVINCIA,142.0,-12.0756,-77.0433,70.2,71530,23794,75359
8,27,1300,LIMA,LIMA,MAGDALENA DEL MAR,LIMA PROVINCIA,90.0,-12.0917,-77.0672,70.2,59059,19239,60290
9,26,1296,LIMA,LIMA,LINCE,LIMA PROVINCIA,150.0,-12.0844,-77.0303,70.2,54119,18193,54711


In [20]:
df_copia=df_original

#### 4.2 TRABAJO CON FOURSQUARE

In [4]:
!pip install requests
import requests
url = "https://api.foursquare.com/v3/places/search"
headers = {"Accept": "application/json"}
response = requests.get(url, headers=headers)
print(response.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Header overflow


In [ ]:
# Definir la versión y credenciales de FourSquare
CLIENT_ID = 'RQAESOZ2QMHMT1HXIZBVD4RY2B4DQRUYLL5P2RWCMRCUKM0M' #  ID de Foursquare (CLLG)
CLIENT_SECRET = '45YQXLRPSIUWQVROG3IIC0PDVLMLD5FE4WX4E3LBOWNWMS5W' # Secreto de Foursquare
MI_AUTORIZACION= "fsq3qyO+ASIY0qnySGSsFGpdlgk2qTPwb5lY+ysIhpmq7aI="
VERSION = '20211811' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('MI_AUTORIZACION:'+ MI_AUTORIZACION)

Your credentails:
CLIENT_ID: RQAESOZ2QMHMT1HXIZBVD4RY2B4DQRUYLL5P2RWCMRCUKM0M
CLIENT_SECRET:45YQXLRPSIUWQVROG3IIC0PDVLMLD5FE4WX4E3LBOWNWMS5W
MI_AUTORIZACION:fsq3qyO+ASIY0qnySGSsFGpdlgk2qTPwb5lY+ysIhpmq7aI=


In [ ]:
# Ubicamos los datos de uno de los barrios de los municipios seleccionados.
df_distrito=df_original
print('Distrito elegido para prueba en Foursquare es:' , df_distrito.loc[1, 'distrito'])
distrito_latitud = df_distrito.loc[0,'latitud'] # latitud del distrito 
distrito_longitud = df_distrito.loc[0,'longitud'] # longitud del distrito
distrito_name = df_distrito.loc[0,'distrito'] # nombre del distrito
print('Latitud y longitud de {} son {}, {}.'.format(distrito_name,distrito_latitud,distrito_longitud))
llvalor=(distrito_latitud,distrito_longitud)
print(llvalor)

Distrito elegido para prueba en Foursquare es: LA MOLINA
Latitud y longitud de SANTIAGO DE SURCO son -12.145, -77.005.
(-12.145, -77.005)


In [5]:
!pip install requests

import requests

url = "https://api.foursquare.com/v3/places/search?radius=500&limit=25&ll=-12.145,-77.005"

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3qyO+ASIY0qnySGSsFGpdlgk2qTPwb5lY+ysIhpmq7aI="
}

response = requests.get(url, headers=headers)

print(response.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
{"results":[{"fsq_id":"4d98b67f0caaa1432a8cb3b3","categories":[{"id":13095,"name":"Cajun / Creole Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/cajun_","suffix":".png"}}],"chains":[],"distance":11,"geocodes":{"main":{"latitude":-12.144925,"longitude":-77.004926}},"link":"/v3/places/4d98b67f0caaa1432a8cb3b3","location":{"admin_region":"Lima","country":"PE","cross_street":"","formatted_address":"Lima","locality":"Lima","postcode":"","region":"Lima"},"name":"Meson de Surco","related_places":{},"timezone":"America/Lima"},{"fsq_id":"4dc189581f6e7441cf503a0f","categories":[{"id":16041,"name":"Plaza","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_","suffix":".png"}}],"chains":[],"distance":79,"geocodes":{"main":{"latitude":-12.145422,"longitude":-77.00466},"roof":{"latitude":-12.145422,"longitude":-77.00466}},"link":"/v3/places/4dc189581

El soporte con foursquare no funciona para la version 2, se intenta con la version 3 arroja error.

In [14]:
## segunda parte
## cargar el geopy 
## verifiqe si esta geopy cargado en conda sino instale manualmente
# conda install -c conda-forge geopy ** y le da Y
from geopy.geocoders import Nominatim
# carga folium
!pip install folium
import folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
# Buscamos las coordenadas para realizar el mapa
address = 'Lima, Lima'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas de '+address,' son {}, {}.'.format(latitude,longitude))

Las coordenadas de Lima, Lima  son -12.0621065, -77.0365256.


In [18]:
# hacemos el  mapa de la ciudad de Lima (Perú)
mapa_lima = folium.Map(location=[latitude, longitude], zoom_start = 10)
mapa_lima

In [21]:
# ahora ponemos los marcadores de los distritos
for lat,lng, borough, neighborhood in zip(df_copia['latitud'],df_copia['longitud'],df_copia['provincia_x'],df_copia['distrito']):
    label= 'distrito {}, provincia {}'.format(neighborhood,borough)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup= label,
        color= 'red',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(mapa_lima)

mapa_lima